In [1]:
import pandas as pd
import nltk
from nltk import punkt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
from tqdm import tqdm
pd.options.display.max_columns = 999

In [2]:
lemmatizer = WordNetLemmatizer() 

In [3]:
tokenizer = sent_tokenize

In [4]:
df = pd.read_csv('cleanedManhattanfull.csv')

In [5]:
import string 
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [ ]:
df.head()

## Sample Run

In [ ]:
sample = df.iloc[0:1]

In [ ]:
sample

In [ ]:
sampledrop = ['review_'+str(x) for x in range(125,140)]

In [ ]:
sampledrop

In [ ]:
sample.drop(columns=sampledrop, axis=1, inplace=True)

In [ ]:
sample

In [ ]:
sample.drop(columns=['Unnamed: 0', 'price', 'rating', 'restaurant_id', 'review_count'], axis=1, inplace=True)

In [ ]:
sample

In [ ]:
rva = ['review_'+ str(x+1) for x in range(124)]

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df = df.fillna(value=str('none'))

In [ ]:
df.drop(columns=rva, inplace=True)

In [ ]:
sample.drop(columns=rva, inplace=True)

In [ ]:
sample.head()

In [ ]:
sample.drop(columns=['restaurant_name'], inplace=True)

In [ ]:
sample.head()

In [ ]:
for y in rva:
    sample[y] = sample[y].apply(remove_punctuations)

In [ ]:
for y in rva:
    sample[y] = sample[y].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in (stop)]))

In [ ]:
sample.head()

In [ ]:
for y in rva:
    sample[y] = sample[y].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos = 'r') for word in x.split()]))

In [ ]:
df.head()

In [ ]:
for y in tqdm(rva):
    sample['token_'+y]= sample[y].apply(tokenizer)

In [ ]:
sample.head()

In [ ]:
sample.values

In [ ]:
import numpy as np
tokenslist = sample.values.tolist()

In [ ]:
print(len(tokenslist[0][10]))

In [ ]:
tokenslist

In [ ]:
#FUNCTION TO FLATTEN TOKEN LIST

def flatten( alist ):
     newlist = []
     for item in alist:
         if isinstance(item, list):
             newlist = newlist + flatten(item)
         else:
             newlist.append(item)
     return newlist

In [ ]:
flattoken = flatten(tokenslist)

In [ ]:
print(len(flattoken))

In [ ]:
flattoken

In [ ]:
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(sample.values[0])

doc_term_matrix = [dictionary.doc2bow(doc) for doc in sample.values[0]]

In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=100)

In [ ]:
print(ldamodel.print_topics(num_topics=10, num_words=10))

## Full Run

In [ ]:
rva

In [6]:
rva = ['review_'+ str(x+1) for x in range(139)]

In [7]:
df = df.fillna(value=str('none'))

In [8]:
df.drop(columns=['Unnamed: 0', 'price', 'rating', 'restaurant_id', 'review_count'], axis=1, inplace=True)

In [ ]:
#for y in rva:
#    df[y] = df[y].apply(remove_punctuations)

In [9]:
for y in rva:
    df[y] = df[y].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.lower() not in (stop)]))

In [10]:
for y in tqdm(rva):
    df[y] = df[y].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos = 'r') for word in x.split()]))

100%|██████████| 139/139 [00:22<00:00,  6.15it/s]


In [11]:
for y in tqdm(rva):
    df['token_'+y]= df[y].apply(tokenizer)

100%|██████████| 139/139 [00:36<00:00,  3.76it/s]


In [ ]:
df.head()

In [12]:
df.drop(columns=rva, inplace=True)

In [13]:
df.drop(columns=['restaurant_name'], axis=1, inplace=True)

In [ ]:
df.head()

In [14]:
tokenslist = df.values.tolist()

In [ ]:
len(tokenslist)

In [ ]:
type(tokenslist)

In [ ]:
df.head()

In [ ]:
df['token_review_129'][0]

In [15]:
temp = []
for row in tokenslist:
    if len(row) > 1:
        temp.extend(row)
#remove below line for full run        
# temp = temp[0:50] + temp[1000:1100]

In [ ]:
temp[129]

In [16]:
def deleting(list_1,del_name):
    for sub_list in tqdm(list_1):
        if del_name in sub_list:
            list_1.remove(sub_list)
    return list_1

deleting(temp, 'none')


 94%|█████████▍| 117378/124544 [00:10<00:00, 11460.64it/s]


[['reservation saturday night dinner.',
  'everything ordered incredible.',
  'started half dozen oysters tuna tartare.',
  'oysters fresh tasty.',
  "tuna tartare really yummy, came served bread would've enjoyed plantain chips too.",
  'entrees, split scallops grilled octopus.',
  'flavorful cooked perfection.',
  'want come back happy hour!'],
 ['new go-to place seafood, still liked it.came sunday night -- made reservation yelp -- pretty packed.',
  'saw people turned away, definitely make reservation case.my friend went straight entrees: surf turf (hangar steak grilled shrimp sugar snap peas, fingerling potatoes, & balsamic reduction) scallops special (deep diver scallops mushroom brown rice risotto).',
  'dishes delish think reasonably priced.',
  'looks like menu change often, definitely change.',
  "also love serve 7 entrees, can't get overwhelmed offerings -- limited menu helps get entrees faster!i think dishes reasonably portioned -- filling, enough satisfied meal.cailin's reco

### Full Run Corpus

In [ ]:
temp

In [19]:
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(temp)
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=8000)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in temp]


### Full Run LDA

In [20]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, per_word_topics=True, id2word = dictionary, passes=100, random_state=12)

In [21]:
ldamodel.print_topics(num_topics=10, num_words=10)

[(0,
  '0.075*"disappointed." + 0.073*"highly recommend!" + 0.070*"amazing!" + 0.066*"food good." + 0.033*"service excellent." + 0.033*"yes." + 0.025*"great place!" + 0.023*"okay." + 0.018*"yummy!" + 0.018*"definitely come back!"'),
 (1,
  '0.818*"none" + 0.014*"yum!" + 0.010*"highly recommended!" + 0.009*"food amazing!" + 0.008*"lol." + 0.007*"staff friendly." + 0.004*"loved place!" + 0.004*"delicious food." + 0.004*"impressed." + 0.003*"incredible."'),
 (2,
  '0.077*"wow!" + 0.065*"." + 0.054*"back!" + 0.036*"excellent." + 0.031*"no." + 0.028*"really good." + 0.026*"enjoy!" + 0.024*"why?" + 0.023*"great place." + 0.022*"regret it."'),
 (3,
  '0.177*"good." + 0.108*"delicious!" + 0.023*"it." + 0.020*"absolutely delicious." + 0.018*"everything delicious!" + 0.018*"bad." + 0.017*"love it." + 0.017*"definitely recommend." + 0.016*"yes!" + 0.015*"cool."'),
 (4,
  '0.079*"love place!" + 0.064*"service great." + 0.052*"food delicious." + 0.047*":)" + 0.046*"food amazing." + 0.037*"great." +

In [22]:
from gensim.test.utils import datapath
# Save model to disk.

ldamodel.save('ldabest.model')

In [23]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel2 = Lda(doc_term_matrix, num_topics=10, per_word_topics=True, id2word = dictionary, passes=100, random_state=15)

In [24]:
ldamodel2.print_topics(num_topics=10, num_words=20)

[(0,
  '0.084*"good!" + 0.048*"food great." + 0.045*"great." + 0.037*"everything delicious." + 0.035*"yum." + 0.027*"great place!" + 0.023*"food excellent." + 0.022*"worth it." + 0.018*"great service!" + 0.018*"definitely coming back." + 0.018*"good food." + 0.017*"great food!" + 0.015*"delicious food." + 0.014*"highly recommend" + 0.014*"place awesome." + 0.013*"we\'ll back!" + 0.012*"coming back." + 0.012*"wonderful." + 0.011*"staff friendly attentive." + 0.011*"enjoy."'),
 (1,
  '0.164*"good." + 0.073*"amazing." + 0.059*"food good." + 0.052*"food delicious." + 0.029*"excellent." + 0.023*"really good." + 0.018*"meh." + 0.018*"regret it." + 0.017*"definitely come back." + 0.016*"pretty good." + 0.015*"definitely recommend." + 0.015*"yes!" + 0.014*"must try!" + 0.013*"service amazing." + 0.012*"check." + 0.012*"service okay." + 0.011*"really?" + 0.011*"incredible." + 0.010*"so." + 0.009*"i\'ll definitely back!"'),
 (2,
  '0.578*"!" + 0.048*"amazing!" + 0.033*"back!" + 0.016*"great!" + 

In [30]:
from gensim.test.utils import datapath
ldamodel2.save('ldabest2.model')

In [25]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel3 = Lda(doc_term_matrix, num_topics=10, per_word_topics=True, id2word = dictionary, passes=100, random_state=15)

In [26]:
ldamodel3.print_topics(num_topics=10, num_words=20)

[(0,
  '0.084*"good!" + 0.048*"food great." + 0.045*"great." + 0.037*"everything delicious." + 0.035*"yum." + 0.027*"great place!" + 0.023*"food excellent." + 0.022*"worth it." + 0.018*"great service!" + 0.018*"definitely coming back." + 0.018*"good food." + 0.017*"great food!" + 0.015*"delicious food." + 0.014*"highly recommend" + 0.014*"place awesome." + 0.013*"we\'ll back!" + 0.012*"coming back." + 0.012*"wonderful." + 0.011*"staff friendly attentive." + 0.011*"enjoy."'),
 (1,
  '0.164*"good." + 0.073*"amazing." + 0.059*"food good." + 0.052*"food delicious." + 0.029*"excellent." + 0.023*"really good." + 0.018*"meh." + 0.018*"regret it." + 0.017*"definitely come back." + 0.016*"pretty good." + 0.015*"definitely recommend." + 0.015*"yes!" + 0.014*"must try!" + 0.013*"service amazing." + 0.012*"check." + 0.012*"service okay." + 0.011*"really?" + 0.011*"incredible." + 0.010*"so." + 0.009*"i\'ll definitely back!"'),
 (2,
  '0.578*"!" + 0.048*"amazing!" + 0.033*"back!" + 0.016*"great!" + 

In [32]:
from gensim.test.utils import datapath
ldamodel3.save('ldabest3.model')

In [27]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel4 = Lda(doc_term_matrix, num_topics=9, per_word_topics=True, id2word = dictionary, passes=150, random_state=5)

In [28]:
ldamodel4.print_topics(num_topics=9, num_words=20)

[(0,
  '0.088*"love place." + 0.069*"good!" + 0.061*"wow!" + 0.048*"service good." + 0.028*"great service." + 0.024*"highly recommended." + 0.021*"staff friendly." + 0.018*"worth it." + 0.018*"food great!" + 0.018*"great place." + 0.018*"place amazing!" + 0.017*"return." + 0.016*"food really good." + 0.015*"go." + 0.014*"yes!" + 0.013*"would recommend." + 0.013*"disappoint." + 0.011*"know." + 0.011*"great spot." + 0.010*"coming back."'),
 (1,
  '0.119*"good." + 0.048*"disappointed." + 0.046*"service great." + 0.042*"definitely back!" + 0.034*":)" + 0.033*"yum!" + 0.031*"back!" + 0.029*"food great." + 0.027*"loved it." + 0.027*"great." + 0.022*"highly recommended!" + 0.019*"lol." + 0.016*"thank you!" + 0.016*"enjoy!" + 0.015*"place great!" + 0.013*"friendly staff." + 0.011*"definitely recommend." + 0.011*"yummy." + 0.011*"disappointed!" + 0.010*"would definitely go back."'),
 (2,
  '0.080*"highly recommend!" + 0.061*"back." + 0.027*"great place!" + 0.021*"love it!" + 0.020*"yummy!" + 0.

### Full Run Multi-Core LDA

In [ ]:
lda_modelmulti = gensim.models.LdaMulticore(doc_term_matrix, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [ ]:
lda_modelmulti.print_topics(num_topics=5, num_words=20)

In [ ]:
lda_modelmulti2 = gensim.models.LdaMulticore(doc_term_matrix, num_topics=15, id2word=dictionary, passes=2, workers=2)

In [ ]:
lda_modelmulti2.print_topics(num_topics=15, num_words=20)

In [ ]:
words_to_ignore = ['lamb', 'pork', 'beef', 'chicken', 'duck', 'oyster', 'clam', 'clams', 'oysters', 'cheese', 'eggs', 'bacon', 'pizza']

In [ ]:
lda_modelmulti3 = gensim.models.LdaMulticore(doc_term_matrix, num_topics=25, alpha='asymmetric', id2word=dictionary, passes=30, workers=2)

In [ ]:
lda_modelmulti3.print_topics(num_topics=25, num_words=20)